In [6]:
import numpy as np
import random # Import the random module

# Example distance matrix for 5 cities
distance_matrix = np.array([
    [0, 2, 9, 10, 7],
    [1, 0, 6, 4, 3],
    [15, 7, 0, 8, 11],
    [6, 13, 12, 0, 5],
    [10, 4, 8, 6, 0]
])

# Step 1: Initialize population with random routes
def initialize_population(num_cities, population_size):
    return [np.random.permutation(num_cities) for _ in range(population_size)]

# Step 2: Define the fitness function (total route distance)
def fitness(route, distance_matrix):
    total_distance = sum(distance_matrix[route[i], route[i+1]] for i in range(len(route)-1))
    total_distance += distance_matrix[route[-1], route[0]] # Return to start
    return 1 / total_distance  # Inverse of distance as fitness



# Step 3: Selection using random.sample
def selection(population, fitness_scores):
    # Return two distinct routes selected randomly from the population
    return random.sample(population, 2)


# Step 4: Crossover using Order Crossover (OX)
def crossover(parent1, parent2, crossover_rate):
    if np.random.rand() > crossover_rate:
        return parent1, parent2  # No crossover

    # Example of Order Crossover (OX) implementation:
    size = len(parent1)
    start, end = sorted(np.random.choice(range(size), 2, replace=False))

    # Initialize offspring with -1 placeholders
    offspring1, offspring2 = [-1] * size, [-1] * size

    # Copy the crossover section from each parent to the respective offspring
    offspring1[start:end] = parent1[start:end]
    offspring2[start:end] = parent2[start:end]

    # Fill the remaining positions with the other parent's genes
    fill_positions(offspring1, parent2, start, end)
    fill_positions(offspring2, parent1, start, end)

    return offspring1, offspring2

# Helper function for crossover to fill positions
def fill_positions(offspring, parent, start, end):
    size = len(parent)
    current_pos = end
    for gene in parent:
        if gene not in offspring:
            if current_pos == size:
                current_pos = 0
            offspring[current_pos] = gene
            current_pos += 1

# Step 5: Mutation by swapping two cities
def mutate(route, mutation_rate):
    if np.random.rand() < mutation_rate:
        i, j = np.random.choice(len(route), 2, replace=False)
        route[i], route[j] = route[j], route[i]
    return route

# Main Genetic Algorithm loop
def genetic_algorithm(distance_matrix, num_cities, population_size, generations, crossover_rate, mutation_rate):
    population = initialize_population(num_cities, population_size)

    for gen in range(generations):
        # Evaluate fitness for each route in the population
        fitness_scores = [fitness(route, distance_matrix) for route in population]

        # Generate new population
        new_population = []
        while len(new_population) < population_size:
            # Step 3: Select two parents
            parent1, parent2 = selection(population, fitness_scores)

            # Step 4: Perform crossover
            offspring1, offspring2 = crossover(parent1, parent2, crossover_rate)

            # Step 5: Mutate the offspring
            mutate(offspring1, mutation_rate)
            mutate(offspring2, mutation_rate)

            # Add offspring to the new population
            new_population.extend([offspring1, offspring2])

        # Update population with the new generation
        population = new_population

    # Find and return the best route
    best_route = min(population, key=lambda route: 1/fitness(route, distance_matrix))
    return best_route

# Set parameters
population_size = 10
generations = 50
crossover_rate = 0.6
mutation_rate = 0.1
num_cities = len(distance_matrix)

# Run the Genetic Algorithm
best_route = genetic_algorithm(
    distance_matrix,
    num_cities,
    population_size,
    generations,
    crossover_rate,
    mutation_rate
)

# Output the best route and its total distance
total_distance = 1 / fitness(best_route, distance_matrix)
print("Best Route:", best_route)
print("Total Distance:", total_distance)


Best Route: [4, 0, 1, 2, 3]
Total Distance: 31.0
